In [1]:
!pip install pyspark

In [2]:
    # Install OpenJDK 8
    !apt-get update
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null

    # Set JAVA_HOME environment variable
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,202 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,848 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubu

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

**Note:** The `PySparkRuntimeError` in cell `7AkR3HpzifV6` occurs because a SparkSession has already been initialized. You do not need to run that cell again. You can proceed with using the existing `spark` session for your PySpark tasks.

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv('/content/HousingData.csv',inferSchema=True, header =True)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
dataset.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MEDV: double (nullable = true)



In [ ]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","MEDV")

finalized_data.show()

+--------------------+----+
|          Attributes|MEDV|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.02985,0.0,2.18...|28.7|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
|[1.25179,0.0,8.14...|13.6|
|[0.85204,0.0,8.14...|19.6|
|[1.23247,0.0,8.14...|15.2|
|[0.98843,0.0,8.14...|14.5|
+--------------------+----+
only showing top 20 rows



In [ ]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'MEDV')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|MEDV|        prediction|
+--------------------+----+------------------+
|[0.02498,0.0,1.89...|16.5| 22.10129862249039|
|[0.02729,0.0,7.07...|34.7|30.715564072178132|
|[0.03041,0.0,5.19...|18.5| 19.24916158802127|
|[0.03359,75.0,2.9...|34.9|35.090202812520324|
|[0.03427,0.0,5.19...|19.5|20.294356198573276|
|[0.03738,0.0,5.19...|20.7|21.877543003817152|
|[0.04113,25.0,4.8...|28.0|28.938533369084976|
|[0.04297,52.5,5.3...|24.8|27.476302384705072|
|[0.04379,80.0,3.3...|19.4|25.703991635414415|
|[0.04462,25.0,4.8...|23.9|26.934836588558767|
|[0.05059,0.0,4.49...|23.9|25.069244640546774|
|[0.05644,40.0,6.4...|32.4| 37.13453054468183|
|[0.07886,80.0,4.9...|37.3| 35.32374745279883|
|[0.08265,0.0,13.9...|23.9| 26.15065757765796|
|[0.09378,12.5,7.8...|21.7|21.159141597610457|
|[0.10469,40.0,6.4...|33.2| 36.84620541675951|
|[0.11027,25.0,5.1...|22.2| 23.77022399236281|
|[0.1146,20.0,6.96...|24.4|28.632436094718344|
|[0.11747,12.

In [ ]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.0548, 0.0492, 0.0272, 3.3932, -16.9082, 4.4558, -0.0172, -1.4653, 0.2577, -0.0117, -0.8427, 0.0128, -0.4522])
The Intercept of the model is : 28.868112


In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
rf = RandomForestRegressor(featuresCol="Attributes", labelCol = 'MEDV')
regressor_train = rf.fit(train_data)
#To predict the prices on testing set
predictions = regressor_train.transform(test_data)

#Predict the model
#pred_test.select("prediction", "label", "features").show(5)

# Select example rows to display.
predictions.select("prediction", "MEDV", "Attributes").show(5)

evaluator = RegressionEvaluator(
    labelCol="MEDV", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+------------------+----+--------------------+
|        prediction|MEDV|          Attributes|
+------------------+----+--------------------+
|25.034175586120124|16.5|[0.02498,0.0,1.89...|
| 34.82439814914301|34.7|[0.02729,0.0,7.07...|
|20.785931196138982|18.5|[0.03041,0.0,5.19...|
| 36.04452892730793|34.9|[0.03359,75.0,2.9...|
| 21.49342202611614|19.5|[0.03427,0.0,5.19...|
+------------------+----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 3.2025


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="MEDV", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 4.021
MSE: 16.171
MAE: 2.848
r2: 0.712
